In [1]:
import torch

# پارامترهای ساده
T, B, N = 2, 1, 3                # توالی تاریخی ۲، یک بَچ، ۳ گره
D, F = 5, 1                      # ابعاد ورودی = ۵، ابعاد خروجی = ۱
hidden_dim, heads = 4, 2         # H=4 (باید قابل تقسیم بر heads باشد)
dist_adj = torch.eye(N)          # گراف فاصله ساده: ماتریس همانی

# دوتا تست: هم بدون DAGMA و هم با DAGMA دستی
x_with_pre = torch.randn(T+1, B, N, D)  # شکل (3,1,3,5)
print("x_with_pre.shape:", x_with_pre.shape)


x_with_pre.shape: torch.Size([3, 1, 3, 5])


In [2]:
from nns.case.dag_gen_gru import DagGenGRUg4s2v1

# مدل را با use_dagma=False بساز
model_nd = DagGenGRUg4s2v1(
    num_nodes=N, in_feats_dim=D, out_feats_dim=F,
    hidden_dim=hidden_dim, num_layers=1, num_heads=heads,
    feats_layers=1, dist_adj=dist_adj, agg_feats='ori',
    node_norm=False, use_norm=False,
    use_dagma=False, use_lib_dagma=False  # بدون هیچ پروجکتوری
)

# فراموش نکن اگر در GPU هستی model_nd.to(device)
model_nd.eval()
with torch.no_grad():
    graphs_nd, reconst_nd = model_nd(x_with_pre, gen_graph_only=False)

print("Without DAGMA → graphs:", graphs_nd.shape)
print("Without DAGMA → reconst:", reconst_nd.shape)


Without DAGMA → graphs: torch.Size([2, 1, 2, 3, 3])
Without DAGMA → reconst: torch.Size([2, 1, 3, 1])


In [3]:
from nns.case.dag_gen_gru import DagGenGRUg4s2v1

# مدل با پروجکتور دستی DAGMA
model_hd = DagGenGRUg4s2v1(
    num_nodes=N, in_feats_dim=D, out_feats_dim=F,
    hidden_dim=hidden_dim, num_layers=1, num_heads=heads,
    feats_layers=1, dist_adj=dist_adj, agg_feats='ori',
    node_norm=False, use_norm=False,
    use_dagma=True, use_lib_dagma=False,  # فقط نسخه دستی
    dagma_mu=1.0, dagma_alpha=1.0, dagma_steps=2
)
model_hd.eval()
with torch.no_grad():
    graphs_hd, reconst_hd = model_hd(x_with_pre, gen_graph_only=False)

print("Hand‐written DAGMA → graphs:", graphs_hd.shape)
print("Hand‐written DAGMA → reconst:", reconst_hd.shape)


Hand‐written DAGMA → graphs: torch.Size([2, 1, 2, 3, 3])
Hand‐written DAGMA → reconst: torch.Size([2, 1, 3, 1])


In [4]:
from nns.case.dag_gen_gru import DagGenGRUg4s2v1

# مدل با پروجکتور دستی DAGMA
model_hd = DagGenGRUg4s2v1(
    num_nodes=N, in_feats_dim=D, out_feats_dim=F,
    hidden_dim=hidden_dim, num_layers=1, num_heads=heads,
    feats_layers=1, dist_adj=dist_adj, agg_feats='ori',
    node_norm=False, use_norm=False,
    use_dagma=True, use_lib_dagma=True,  # نسخه کتابخوانه ای
    dagma_mu=1.0, dagma_alpha=1.0, dagma_steps=2
)
model_hd.eval()
with torch.no_grad():
    graphs_hd, reconst_hd = model_hd(x_with_pre, gen_graph_only=False)

print("Hand‐written DAGMA → graphs:", graphs_hd.shape)
print("Hand‐written DAGMA → reconst:", reconst_hd.shape)


Hand‐written DAGMA → graphs: torch.Size([2, 1, 2, 3, 3])
Hand‐written DAGMA → reconst: torch.Size([2, 1, 3, 1])


In [5]:
graphs, _ = model_hd(x_with_pre, gen_graph_only=True)
print("graphs shape:", graphs.shape)  # Expect (T, B, 2, N, N)


graphs shape: torch.Size([2, 1, 2, 3, 3])


In [6]:
from nns.case.pred_blocks import CascadeGCNv2d4

pred_model = CascadeGCNv2d4(
    num_nodes=N,        # تعداد گره‌های گراف (مثلاً 3)
    input_dim=D,        # بعد ویژگی ورودی در هر گره (مثلاً 5)
    output_dim=F,       # بعد خروجی (مثلاً 1 برای سرعت/جریان)
    hidden_dim=16,      # سایز فضای نهان داخل GCN
    dist_adj=dist_adj,  # ماتریس فاصله ثابت (برای اجزای گراف پیش‌فرض)
    num_layers=2,       # تعداد لایه‌های cascade در هر بلوک
    seq_len=T,          # طول توالی ورودی (مثلاً 2)
    horizon=12       # طول توالی پیش‌بینی (مثلاً 2 یا 12)
)


In [7]:
x_hist = x_with_pre[1:]

preds = pred_model(x_hist, graphs)
print("preds shape:", preds.shape)  # Expect (horizon, B, N, out_dim)


preds shape: torch.Size([12, 1, 3, 1])


In [8]:
print(preds)             # خود مقادیر
print(preds.min(), preds.max())


tensor([[[[ 0.0777],
          [ 0.0026],
          [ 0.3642]]],


        [[[-1.5881],
          [-0.9944],
          [-1.1983]]],


        [[[ 1.0413],
          [ 1.3954],
          [ 0.8983]]],


        [[[-0.6704],
          [ 1.0071],
          [ 0.5868]]],


        [[[-0.1207],
          [-0.4232],
          [-0.4399]]],


        [[[ 0.1336],
          [-0.6387],
          [-1.1277]]],


        [[[ 1.6852],
          [ 0.9076],
          [ 0.4550]]],


        [[[ 0.5845],
          [ 0.7953],
          [ 0.2825]]],


        [[[ 0.9248],
          [-0.3479],
          [-0.9326]]],


        [[[-0.0794],
          [ 0.8175],
          [ 0.7715]]],


        [[[-0.0404],
          [-0.8377],
          [-0.8250]]],


        [[[ 1.0412],
          [ 1.4962],
          [ 0.2766]]]], grad_fn=<PermuteBackward0>)
tensor(-1.5881, grad_fn=<MinBackward1>) tensor(1.6852, grad_fn=<MaxBackward1>)


In [14]:
from utils.metrics_lib import All_Metrics
y_true = torch.randn_like(preds)
mae, rmse, mape = All_Metrics(preds, y_true, 0.1, 0.1 )
print(f"MAE: {mae:.4f}, RMSE: {rmse:.4f}, MAPE: {mape*100:.2f}%")


MAE: 1.0658, RMSE: 1.4096, MAPE: 93.08%


In [ ]:
import os
import yaml
import torch
from utils.path_utils import get_model_config_path, base_dir
from nns.case.supervisor_pred import CASECausalPred

# ۱) مسیر نسبی را که تابع کمکت ساخته بگیر
rel_path = get_model_config_path('pred', model_dir_name='models')
#    مثال: "data/models/pred/pred_la.yaml"

# ۲) آن را با base_dir ترکیب کن
cfg_path = os.path.join(base_dir, rel_path)
print("loading config from:", cfg_path)
#    باید چیزی شبیه:
#    loading config from: /.../DCGCN_Dagma/data/models/pred/pred_la.yaml

# ۳) فایل YAML را باز و بخوان
with open(cfg_path, 'r') as f:
    config = yaml.safe_load(f)

# ۴) به config کلید base_dir بده
config['base_dir'] = base_dir

# ۵) ساخت شیء سوپروایزر
supervisor = CASECausalPred(**config)

# ۶) (اختیاری) اگر GPU داری:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# معمولاً داخل Supervisor خودش مدل را به device می‌فرستد.

# ۷) شروع آموزش
supervisor.train()

# ۸) اجرا روی داده‌ی تست و گرفتن نتایج
results = supervisor.test()
print("Test losses per horizon:\n", results['loss'])

# ۹) ذخیره و لاگ کردن هایپرپارامترها و نتایج
supervisor.test_and_log_hparms()


In [16]:


supervisor = CascadeGCNv2d4(**config)
supervisor.train()                 # آموزش causal generator + prediction
results = supervisor.test()        # تست نهایی
supervisor.test_and_log_hparms()   # لاگ‌گیری hparam + metrics


FileNotFoundError: [Errno 2] No such file or directory: 'D:/Code_Files/Pycharm/Traffic_CASE\\data/models/pred\\pred_la.yaml'